# Step 1: Real-Time Feature Computation

This notebook is part of a demo showcasing a real-time fraud detection pipeline, utilizing Feldera for feature computation and Hopsworks as the feature store.

![Real-time feature engineering pipeline using Feldera and Hosworks](./architecture.png)


## Step 1.1. Create Hopsworks feature groups

In [15]:
import hopsworks
import hsfs
from hsfs.feature import Feature
import json
import datetime
from feldera import FelderaClient, PipelineBuilder

client = FelderaClient("http://localhost:8080")

project = hopsworks.login()
hostname = project.get_url().removeprefix("https://").split(":", 1)[0]

kafka_api = project.get_kafka_api()

fs = project.get_feature_store()

name = "cc_trans_aggs_fg"
kafka_topic = f"{project.name}_onlinefs"
aggs_topic=f"{project.name}_{name}"

cc_trans_fg = fs.get_feature_group(name="credit_card_transactions", version=1)
card_details_fg = fs.get_feature_group(name="card_details", version=1)


# WINDOWED - frequency of transactions and other metrics in the span of a few hours, modeled as hopping window aggregates.
windowed_fg = fs.get_or_create_feature_group(
    name=name,
    primary_key=["cc_num"],
    online_enabled=True,
    version=1,
    event_time="event_time",
    # topic_name=aggs_topic,    
    stream=True,
    features=[        
        Feature("cc_num", type="string"),
        Feature("account_id", type="string"),
        Feature("bank_id", type="string"),
        Feature("num_trans_last_10_mins", type="bigint"),
        Feature("sum_trans_last_10_mins", type="double"),
        Feature("num_trans_last_hour", type="bigint"),
        Feature("sum_trans_last_hour", type="double"),
        Feature("num_trans_last_day", type="bigint"),
        Feature("sum_trans_last_day", type="double"),
        Feature("num_trans_last_week", type="bigint"),
        Feature("sum_trans_last_week", type="double"),
        Feature("prev_card_present", type="boolean"),
        Feature("prev_ip_transaction", type="string"),
        Feature("prev_ts_transaction", type="timestamp"),
        Feature("event_time", type="timestamp"),
    ],    
)



2025-10-20 20:06:31,181 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-10-20 20:06:31,186 INFO: Initializing external client
2025-10-20 20:06:31,187 INFO: Base URL: https://stagingmain.devnet.hops.works:443


2025-10-20 20:06:31,934 INFO: Python Engine initialized.

Logged in to project, explore it here https://stagingmain.devnet.hops.works:443/p/119


In [16]:
client = hsfs.client

In [19]:
path = client.get_instance()._get_client_key_path()
cert = client.get_instance()._get_client_cert_path()
print(path)
print(cert)

/tmp/stagingmain.devnet.hops.works/jim/client_key.pem
/tmp/stagingmain.devnet.hops.works/jim/client_cert.pem


In [ ]:
try:
    windowed_fg.save()
except Exception as e:
    print(e)

## Load certs in Feldera Container

Feldera expects the certs to be in /tmp/HOPSWORKS_HOST/HOPSWORKS_PROJECT

In [2]:
import subprocess

# Get container ID
container_id = subprocess.check_output(
    ["docker", "ps", "--filter", "ancestor=ghcr.io/feldera/pipeline-manager:latest", "-q"],
    text=True
).strip()

print(f"container_id is {container_id}")
# Run the command inside the container
subprocess.run([
    "docker", "exec", container_id,
    "bash", "-c",
    f"rm -f /tmp/{hostname} && ln -s /opt/{hostname}/{hostname} /tmp/{hostname}"
])


container_id is 7e208e223696


CompletedProcess(args=['docker', 'exec', '7e208e223696', 'bash', '-c', 'rm -f /tmp/stagingmain.devnet.hops.works && ln -s /opt/stagingmain.devnet.hops.works/stagingmain.devnet.hops.works /tmp/stagingmain.devnet.hops.works'], returncode=0)

## Step 1.2. Create Feldera pipeline

We build a Feldera pipeline to transform raw transaction and profile data into features. In Feldera, feature groups are modeled as SQL views. Thus, we create a SQL program with two input tables (TRANSACTIONS and PROFILES), and two output views, one for each feature group.

![Feldera pipeline](./feldera_pipeline.png)

In [3]:
# Create SQL program parameterized by source and sink connnector configurations.



def build_sql(
    transaction_source_config: str, card_details_source_config: str, fs_sink_config: str
) -> str:
    return f"""

    CREATE TABLE credit_card_transactions (
        t_id BIGINT,
        merchant_id VARCHAR,
        ts TIMESTAMP,
        cc_num VARCHAR,
        amount DOUBLE,
        ip_address VARCHAR,
        card_present BOOLEAN
    ) WITH (
        'connectors' = '[{transaction_source_config}]'
    );

    CREATE MATERIALIZED VIEW rolling_aggregates AS
    SELECT
        t.cc_num, 
        t.ts AS event_time, 
        t.ip_address,
        t.card_present,
        SUM(COALESCE(amount, 0)) OVER window_10_minute AS sum_trans_last_10_mins,
        COUNT(amount) OVER window_10_minute AS num_trans_last_10_mins,
        SUM(COALESCE(amount, 0)) OVER window_1_hour AS sum_trans_last_hour,
        COUNT(amount) OVER window_1_hour AS num_trans_last_hour,
        SUM(COALESCE(amount, 0)) OVER window_1_day AS sum_trans_last_day,
        COUNT(amount) OVER window_1_day AS num_trans_last_day,
        SUM(COALESCE(amount, 0)) OVER window_7_day AS sum_trans_last_week,
        COUNT(amount) OVER window_7_day AS num_trans_last_week
    FROM
         credit_card_transactions AS t
    WINDOW
        window_10_minute AS (
            PARTITION BY cc_num
            ORDER BY ts
            RANGE BETWEEN INTERVAL '10' MINUTE PRECEDING AND CURRENT ROW
        ),
        window_1_hour AS (
            PARTITION BY cc_num
            ORDER BY ts
            RANGE BETWEEN INTERVAL '1' HOUR PRECEDING AND CURRENT ROW
        ),
        window_1_day AS (
            PARTITION BY cc_num
            ORDER BY ts
            RANGE BETWEEN INTERVAL '1' DAY PRECEDING AND CURRENT ROW
        ),
        window_7_day AS (
            PARTITION BY cc_num
            ORDER BY ts
            RANGE BETWEEN INTERVAL '7' DAY PRECEDING AND CURRENT ROW
        )
    ;
    
    CREATE TABLE card_details (
        cc_num VARCHAR NOT NULL,
        account_id VARCHAR NOT NULL,
        bank_id VARCHAR NOT NULL,
        cc_expiry_date TIMESTAMP,
        issue_date TIMESTAMP,
        card_type VARCHAR,
        status VARCHAR,
        last_modified TIMESTAMP
    ) WITH (
        'connectors' = '[{card_details_source_config}]'
    );


    CREATE MATERIALIZED VIEW cc_trans_card AS
    SELECT
        ra.*,
        cd.account_id,
        cd.bank_id
    FROM rolling_aggregates AS ra
    LEFT ASOF JOIN card_details AS cd
        MATCH_CONDITION (ra.event_time >= cd.last_modified)
        ON ra.cc_num = cd.cc_num
    ;
    
    CREATE MATERIALIZED VIEW lagged_trans AS
    SELECT
        ctc.*,
        LAG(event_time) OVER 
          (PARTITION BY cc_num ORDER BY event_time ASC) AS prev_ts_transaction,
        LAG(ip_address) OVER 
          (PARTITION BY cc_num ORDER BY ip_address ASC) AS prev_ip_transaction,
        LAG(card_present) OVER 
          (PARTITION BY cc_num ORDER BY card_present ASC) AS prev_card_present
    FROM cc_trans_card AS ctc;
        
    CREATE VIEW cc_trans_aggs_fg
    WITH (
        'connectors' = '[{fs_sink_config}]'
    ) 
    AS 
        SELECT
            cc_num,
            event_time,
            account_id,
            bank_id,
            sum_trans_last_10_mins,
            num_trans_last_10_mins,
            sum_trans_last_hour,
            num_trans_last_hour,
            sum_trans_last_day,
            num_trans_last_day,
            sum_trans_last_week,
            num_trans_last_week,
            prev_ts_transaction, 
            prev_ip_transaction,
            prev_card_present
        FROM lagged_trans
    ;
    """

### Connect Kafka sources and sinks

We use the Kafka topic created during the data prep step as the input for the TRANSACTIONS table. The output views are also connected to the Hopsworks feature store via Kafka. Hopsworks ingests data from Kafka using the Avro format, so we configure Feldera Kafka connectors with Avro schemas generated by Hopsworks for each feature group.

In [4]:
def create_consumer_kafka_config(kafka_config: dict, fg):
    return kafka_config | {
        "topic": fg._online_topic_name,
        "start_from": "earliest",
#     "security.protocol": "SSL",
#     "ssl.ca.pem": "-----BEGIN CERTIFICATE-----TOPSECRET0\n-----END CERTIFICATE-----\n",
#     "ssl.key.pem": "-----BEGIN CERTIFICATE-----TOPSECRET1\n-----END CERTIFICATE-----\n",
#     "ssl.certificate.pem": "-----BEGIN CERTIFICATE-----TOPSECRET2\n-----END CERTIFICATE-----\n"
        
    }

# "config": {
#     "topic": "book-fair-sales",
#     "start_from": "earliest",
#     "bootstrap.servers": "example.com:9092",
#     "security.protocol": "SSL",
#     "ssl.ca.pem": "-----BEGIN CERTIFICATE-----TOPSECRET0\n-----END CERTIFICATE-----\n",
#     "ssl.key.pem": "-----BEGIN CERTIFICATE-----TOPSECRET1\n-----END CERTIFICATE-----\n",
#     "ssl.certificate.pem": "-----BEGIN CERTIFICATE-----TOPSECRET2\n-----END CERTIFICATE-----\n"
# }
    

def create_producer_kafka_config(kafka_config: dict, fg, project):
    return kafka_config | {
        # "topic": f"{project.name}_onlinefs",
        "topic": fg._online_topic_name,
        "auto.offset.reset": "earliest",
        "headers": [
            {
                "key": "projectId",
                "value": str(project.id),
            },
            {
                "key": "featureGroupId",
                "value": str(fg.id),
            },
            {
                "key": "subjectId",
                "value": str(fg.subject["id"]),
            },
        ],
    }

kafka_config = kafka_api.get_default_config()

fs_sink_config = json.dumps(
    {
        "transport": {
            "name": "kafka_output",
            "config": create_producer_kafka_config(kafka_config, windowed_fg, project),
        },
        "format": {
            "name": "avro",
            "config": {"schema": windowed_fg.avro_schema, "skip_schema_id": True},
        },
    }
)

transaction_source_config = json.dumps(
    {
        "transport": {
            "name": "kafka_input",
            "config": create_consumer_kafka_config(kafka_config, cc_trans_fg),
        },
        "format": {
            "name": "avro",
            "config": {"schema": cc_trans_fg.avro_schema, "skip_schema_id": True},
        },
    }
)
card_details_source_config = json.dumps(
    {
        "transport": {
            "name": "kafka_input",
            "config": create_consumer_kafka_config(kafka_config, card_details_fg),
        },
        "format": {
            "name": "avro",
            "config": {"schema": card_details_fg.avro_schema, "skip_schema_id": True},
        },
    }
)

sql = build_sql(transaction_source_config, card_details_source_config, fs_sink_config)
pipeline = PipelineBuilder(client, name="hopsworks_kafka3", sql=sql).create_or_replace()

## Step 1.3. Run the pipeline

In [5]:
# Start the Feldera pipeline.
# Read profile data from the feature store and write it to the `PROFILE` table.
pipeline.start()

import time
time.sleep(900)
# pipeline.stop(force=True)

## Schedule materialization to the offline store

In [2]:
import datetime
windowed_fg.materialization_job.run()


Launching job: cc_trans_aggs_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://stagingmain.devnet.hops.works:443/p/119/jobs/named/cc_trans_aggs_fg_1_offline_fg_materialization/executions
2025-10-20 18:33:01,384 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-10-20 18:33:04,482 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-10-20 18:35:12,978 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-10-20 18:35:13,079 INFO: Waiting for log aggregation to finish.
2025-10-20 18:35:21,433 INFO: Execution finished successfully.


Execution('SUCCEEDED', 'FINISHED', '2025-10-20T16:32:53.000Z', '-op offline_fg_materialization -path hdfs:///Projects/jim/Resources/jobs/cc_trans_aggs_fg_1_offline_fg_materialization/config_1760970810909')

In [ ]:
windowed_fg.materialization_job.schedule(
    cron_expression="0 0 3 * * ? *",
    start_time=datetime.datetime.now(tz=datetime.timezone.utc),
)

In [ ]:
# write_options={
#     "wait_for_online_ingestion":"false",
#     "wait_for_job":"false",
#     "hoodie.streamer.kafka.source.maxEvents":"50000000",
# "hoodie.deltastreamer.source.kafka.auto.offset.reset" : "earliest",
# }

In [ ]:
import hopsworks

project = hopsworks.login()
kafka_api = project.get_kafka_api()
for topic_name in [topic.name for topic in kafka_api.get_topics()]:
    print(f"Found topic: {topic_name}")
    

In [ ]:
topic = kafka_api.get_topic("dowlingj_cc_trans_aggs_fg")
topic.delete()